In [1]:
import pandas as pd
from pyspark.sql.functions import udf, col
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import hour, mean
from pyspark.sql.types import *

# This notebook combines a list of doc ids and topic ids for graph analysis

sc = pyspark.SparkContext()

path_x = '/Users/tiegsti.solomon/Downloads'
path_y = '/Users/tiegsti.solomon'

nodes = sc.textFile(path_y+"/nodes_recovered.csv")
doc_topics = sc.textFile(path_x+"/documents_meta.csv")

In [2]:
# remove header train
rdd_nodes_head = nodes.filter(lambda l: "uuid" in l)
rdd_nodes = nodes.subtract(rdd_nodes_head)

# remove header test
rdd_topics_head = doc_topics.filter(lambda l: "document_id" in l)
rdd_topics = doc_topics.subtract(rdd_topics_head)

rdd_n = rdd_nodes.map(lambda a: a.split(","))
rdd_t = rdd_topics.map(lambda a: a.split(","))

sqlContext = SQLContext(sc)
nodes_ = sqlContext.createDataFrame(rdd_n) # schema err
topics_ = sqlContext.createDataFrame(rdd_t)
topics_.show(5)
nodes_.show(5)

nodes_dfp = nodes_.toPandas()
topics_dfp = topics_.toPandas()
print(len(nodes_dfp))

+-------+----+---+-------------------+
|     _1|  _2| _3|                 _4|
+-------+----+---+-------------------+
|2201694| 798|435|2016-06-18 16:00:00|
| 479008|3012|291|                   |
| 372470|3669|421|2014-09-03 00:00:00|
|2739473|4697|251|                   |
|1357972|6914| 56|2016-04-27 00:00:00|
+-------+----+---+-------------------+
only showing top 5 rows

+------+---+---------+----------+
|    _1| _2|       _3|        _4|
+------+---+---------+----------+
| 11395|  0| 37.09024|-95.712891|
|280649|  1| 37.09024|-95.712891|
|  6923|  0| 37.09024|-95.712891|
|175534|  0|-0.023559| 37.906193|
|221749|  1| 37.09024|-95.712891|
+------+---+---------+----------+
only showing top 5 rows

14074


In [3]:
def convert_id_list(doc_ids):
    return ' '.join(d.split(':')[1] for d in doc_ids.split(' '))

# nodes_dfp = nodes_dfp['_1'].unique()
doc_ids = pd.DataFrame(nodes_dfp, columns = ['_1','_2','_3','_4'])
doc_ids = doc_ids.rename(index=str, columns={'_1':'_1_1'})
topic_ids = pd.DataFrame(topics_dfp, columns = ['_1','_2'])

#merged_dfp = doc_ids.set_index('_1').join(topic_ids.set_index('_1'))

merged_dfp2 = pd.merge(doc_ids, topic_ids,left_on="_1_1",right_on="_1", how='left')

In [4]:
merged_dfp2.drop('_1', axis=1)

,_1_1,_2_x,_3,_4,_2_y
0,11395,0,37.09024,-95.712891,3530
1,280649,1,37.09024,-95.712891,2851
2,6923,0,37.09024,-95.712891,91
3,175534,0,-0.023559,37.906193,2805
4,221749,1,37.09024,-95.712891,632
5,85286,0,37.09024,-95.712891,1256
6,40410,0,37.09024,-95.712891,121
7,167517,1,37.09024,-95.712891,2593
8,13410,0,37.09024,-95.712891,209
9,231127,0,56.130366,-106.346771,3669


In [7]:
merged_dfp2.to_csv("nodes_modified.csv", sep=',')